In [71]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [77]:
DAYSBACK = 365
SQLQUERY = f"""
WITH PC_CD AS (
    SELECT wafer_id||unitcell_X||unitcell_Y AS id, wafer_Id,
        unitcell_X - 1 AS retx, unitcell_Y AS rety,
        dcItem_Value AS PC_ACI_CD_Logic_60CPP
    FROM DMIW.MeasuredRawFact mrf
        INNER JOIN DMIW_SYSTEMS.MeasuredParm mp ON mrf.measuredParmKey = mp.measuredParmKey
        INNER JOIN DMIW_SYSTEMS.Geography g ON mrf.geographyKey = g.geographyKey
        INNER JOIN DMIW_SYSTEMS.MeasuredWafer mw ON mrf.measuredWaferKey = mw.measuredWaferKey
    WHERE Measurement_date >= (current date - {DAYSBACK} days) AND Tech_id = '7HPP' AND Meas_pd_id LIKE '%KU072615' 
        AND meas_Pd_Id <> '' and dcItem_Value is not null 
        AND ucase(dcItem_Name) not in ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
        AND( ucase(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) like '%PC%ACI%CD%(LOGIC%60%' )  
    ORDER BY lot_Id, wafer_Id, unitcell_X - 1, unitcell_Y
),
Sailyield AS(
    SELECT wafer_id||unitcell_X||unitcell_Y AS id, lot_Id, wafer_Id, unitcell_X - 1 as retx, unitcell_Y as rety, DATE(Last_test_date) as date,
        --parmYield AS SAILALL_0p50to1p05_yield
        parmYield AS SAILALL_0p65to1p05_yield
    FROM DMIW.ChipParmFactR cpfr
         INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
         INNER JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
         INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days) AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL11', 'SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        --AND ( UCASE(tp.parm_Label) LIKE '%SAILALL%0P50%PERFECTYIELD%')
        AND ( UCASE(tp.parm_Label) LIKE '%SAILALL%0P65%PERFECTYIELD%')
    ORDER BY lot_Id, wafer_Id, unitcell_X - 1, unitcell_Y
)

SELECT
    s.lot_id, s.wafer_id, s.retx, s.rety, s.date, 
    --sailall_0p50to1p05_yield,
    sailall_0p65to1p05_yield,
    PC_ACI_CD_Logic_60CPP AS "PC ACI CD (logic 60cpp)"
FROM
    Sailyield s
INNER JOIN PC_CD p ON s.id = p.id

"""
df = ibmdata.isdw.query(SQLQUERY)

In [78]:
df.to_csv('~/Downloads/temp.csv')

In [66]:
df

,lot_id,wafer_id,retx,rety,date,sailall_0p65to1p05_yield,pc aci cd (logic 60cpp)
0,AZ6NN.1,AZ6NN-01,1,6,2020-06-29,100.00,99.7
1,AZ6NN.1,AZ6NN-01,1,6,2020-06-29,100.00,99.7
2,AZ6NN.1,AZ6NN-01,3,8,2020-06-29,100.00,98.4
3,AZ6NN.1,AZ6NN-01,3,8,2020-06-29,100.00,98.4
4,AZ6NN.1,AZ6NN-01,4,4,2020-06-29,100.00,100.1
...,...,...,...,...,...,...,...
3115,AZ75R.1,AZ75R-11,7,5,2020-10-08,100.00,98.7
3116,AZ75R.1,AZ75R-11,8,9,2020-10-08,100.00,98.0
3117,AZ75R.1,AZ75R-11,9,6,2020-10-08,100.00,98.3
3118,AZ75R.1,AZ75R-11,10,4,2020-10-08,100.00,98.3


In [79]:
DAYSBACK = 365
SQLQUERY = f"""
WITH sail_cte AS(
    SELECT wafer_id||unitcell_X||unitcell_Y AS id, tw.Calcdefs AS Sail, lot_id,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12) END AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL11', 'SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM')
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        --AND parmYield < 100
        --AND wafer_id = 'AZ6B5-19'
    ORDER BY lot_Id, wafer_Id, parm_Label
),
PC_CD AS (
    SELECT wafer_id||unitcell_X||unitcell_Y AS id, wafer_Id,
        unitcell_X - 1 AS retx, unitcell_Y AS rety,
        dcItem_Value AS PC_ACI_CD_Logic_60CPP
    FROM DMIW.MeasuredRawFact mrf
        INNER JOIN DMIW_SYSTEMS.MeasuredParm mp ON mrf.measuredParmKey = mp.measuredParmKey
        INNER JOIN DMIW_SYSTEMS.Geography g ON mrf.geographyKey = g.geographyKey
        INNER JOIN DMIW_SYSTEMS.MeasuredWafer mw ON mrf.measuredWaferKey = mw.measuredWaferKey
    WHERE Measurement_date >= (current date - {DAYSBACK} days) AND Tech_id = '7HPP' AND Meas_pd_id LIKE '%KU072615' 
        AND meas_Pd_Id <> '' and dcItem_Value is not null 
        AND ucase(dcItem_Name) not in ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
        AND( ucase(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) like '%PC%ACI%CD%(LOGIC%60%' ) 
        AND dcItem_Value < 98
    ORDER BY lot_Id, wafer_Id, unitcell_X - 1, unitcell_Y
)
SELECT 
    s.lot_id, p.wafer_id, p.retx, p.rety, 
    sail, label, yield,
    PC_ACI_CD_Logic_60CPP AS "PC ACI CD (logic 60cpp)"
FROM
    PC_CD p
INNER JOIN sail_cte s ON s.id = p.id
"""
df = ibmdata.isdw.query(SQLQUERY)
df['yield'] = df['yield'].astype(float).round(2)
df['yld_loss'] = 100.00 - df['yield']

In [89]:
df[df['label'].str.contains('block26') | df['label'].str.contains('block28')].wafer_id.unique()

array(['AZ6NN-01', 'AZ6NN-03', 'AZ6NN-15', 'AZ6PC-01', 'AZ6PC-03',
       'AZ6PK-01', 'AZ6PK-04', 'AZ6PK-15', 'AZ6PK-25', 'AZ6R9-15',
       'AZ6R9-17', 'AZ6RA-01', 'AZ6RA-03', 'AZ6RA-04', 'AZ6RA-07',
       'AZ6RA-11', 'AZ6RA-12', 'AZ6RA-15', 'AZ6RA-17', 'AZ6RA-20',
       'AZ6RA-23', 'AZ6RA-25', 'AZ6VB-01', 'AZ6VB-03', 'AZ6VB-04',
       'AZ6VB-07', 'AZ6VB-11', 'AZ6VB-12', 'AZ6VB-15', 'AZ6VB-17',
       'AZ6VB-20', 'AZ6VB-23', 'AZ6W7-03', 'AZ6W7-11', 'AZ6W7-12',
       'AZ6W8-25', 'AZ6WT-15', 'AZ6X4-17', 'AZ6XZ-04', 'AZ6XZ-17',
       'AZ715-23', 'AZ71P-11', 'AZ71P-15', 'AZ71P-25', 'AZ725-01',
       'AZ725-03', 'AZ725-04', 'AZ725-07', 'AZ725-17', 'AZ725-21',
       'AZ725-23', 'AZ725-24', 'AZ725-25', 'AZ72K-01', 'AZ72K-04',
       'AZ75R-11'], dtype=object)

In [96]:
df_agg=df.groupby(['label', 'sail']).mean().sort_values('yld_loss', ascending=False)[0:20].reset_index()

In [75]:
color_map = {'SSL1':'tomato','SSL2':'springgreen','SSL3':'dodgerblue'}
fig = px.bar(df_agg, x='label', y='yld_loss',
            hover_data=['label', 'yld_loss'],
            color = 'sail',
            color_discrete_map=color_map,
            labels={'label':'Label','yld_loss':'yld_loss'+' (%)'})
fig.update_layout(title_text='Sail Yield Loss Pareto for PC CD < 98')
fig.update_layout(xaxis_categoryorder = 'total descending')
fig.show()

In [97]:
df_agg[df_agg['label'].str.contains('block26') | df_agg['label'].str.contains('block28')][0:60]

,wafer_id,label,sail,retx,rety,yield,pc aci cd (logic 60cpp),yld_loss
19,AZ6RA-25,SSL3_0p75_block28_so5,SSL31,6.0,6.0,72.727273,97.018182,27.272727
20,AZ6RA-25,SSL3_0p75_block28_so3,SSL31,6.0,6.0,72.727273,97.018182,27.272727
21,AZ6RA-25,SSL3_0p75_block26_so0,SSL31,6.0,6.0,72.727273,97.018182,27.272727
22,AZ6RA-25,SSL3_0p75_block26_so1,SSL31,6.0,6.0,72.727273,97.018182,27.272727
23,AZ6RA-25,SSL3_0p75_block26_so2,SSL31,6.0,6.0,72.727273,97.018182,27.272727
42,AZ6RA-25,SSL3_0p75_block26_so4,SSL31,6.0,6.0,81.818182,97.018182,18.181818
43,AZ6RA-25,SSL3_0p75_block26_so3,SSL31,6.0,6.0,81.818182,97.018182,18.181818
45,AZ6RA-25,SSL3_0p75_block26_so5,SSL31,6.0,6.0,81.818182,97.018182,18.181818
71,AZ6RA-25,SSL3_0p75_block28_so0,SSL31,6.0,6.0,81.818182,97.018182,18.181818
78,AZ6RA-25,SSL3_0p75_block28_so4,SSL31,6.0,6.0,81.818182,97.018182,18.181818
